# 📘 Performance Optimization: Maximize NLSQ Speed⏱️ **25-35 minutes** | 📊 **Level: ●●○ Intermediate** | 🏷️ **Performance**---

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/imewei/NLSQ/blob/main/examples/notebooks/02_core_tutorials/performance_optimization_demo.ipynb)


In [1]:
# @title Install NLSQ (run once in Colab)
import sys

if 'google.colab' in sys.modules:
    print("Running in Google Colab - installing NLSQ...")
    !pip install -q nlsq
    print("✅ NLSQ installed successfully!")
else:
    print("Not running in Colab - assuming NLSQ is already installed")

Not running in Colab - assuming NLSQ is already installed


## 🎯 What You'll Learn- ✓ **Profile** curve fitting performance- ✓ **Optimize** JAX compilation and JIT- ✓ **Leverage** GPU acceleration- ✓ **Minimize** memory overhead- ✓ **Apply** best practices for speed---

## Setup

In [2]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

In [3]:
import time

import jax
import jax.numpy as jnp
import numpy as np

from nlsq import CurveFit, curve_fit

devices = jax.devices()
print(f"Device: {devices[0].platform}")

Device: gpu


## 1. JAX Compilation Basics**Key insight:** First fit is slow (compilation), subsequent fits are fast

In [4]:
def exponential(x, a, b, c):
    return a * jnp.exp(-b * x) + c

x = np.linspace(0, 10, 1000)
y = 5 * np.exp(-0.5 * x) + 2 + np.random.normal(0, 0.1, 1000)

# First fit (slow - compilation)
start = time.time()
popt1, _ = curve_fit(exponential, x, y, p0=[4, 0.4, 1.5])
t1 = time.time() - start

# Second fit (fast - reuses compiled code)
start = time.time()
popt2, _ = curve_fit(exponential, x, y, p0=[4, 0.4, 1.5])
t2 = time.time() - start

print(f"First fit:  {t1:.4f}s (with compilation)")
print(f"Second fit: {t2:.4f}s (compiled)")
print(f"Speedup: {t1/t2:.1f}x")

INFO:nlsq.curve_fit:Starting curve fit n_params=3 | n_data_points=1000 | method=trf | solver=auto | batch_size=None | has_bounds=False | dynamic_sizing=False


INFO:nlsq.least_squares:Starting least squares optimization method=trf | n_params=3 | loss=linear | ftol=1.0000e-08 | xtol=1.0000e-08 | gtol=1.0000e-08


INFO:nlsq.optimizer.trf:Starting TRF optimization (no bounds) n_params=3 | n_residuals=1000 | max_nfev=None


PERFORMANCE:nlsq.optimizer.trf:Iteration iter=0 | cost=163.72333537348317 | grad_norm=1108.3590 | nfev=1


PERFORMANCE:nlsq.optimizer.trf:Iteration iter=1 | cost=5.333929441542062 | grad_norm=23.3586 | step=4.290687590584987 | nfev=2


...

PERFORMANCE:nlsq.least_squares:Timer: optimization elapsed=0.285437s


INFO:nlsq.least_squares:Convergence reason=`ftol` termination condition is satisfied. | iterations=4 | final_cost=4.9910 | elapsed=0.285s | final_gradient_norm=6.4279e-06


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit elapsed=0.575890s


INFO:nlsq.curve_fit:Curve fit completed total_time=0.5759 | final_cost=9.9820 | covariance_warning=False


First fit:  2.1909s (with compilation)
Second fit: 0.6559s (compiled)
Speedup: 3.3x


## 2. Reusing CurveFit Objects**Best practice:** Reuse CurveFit objects to avoid recompilation

In [5]:
# Create reusable fitter
jcf = CurveFit()

# Multiple fits reuse compiled functions
times = []
for _ in range(10):
    start = time.time()
    popt, _ = jcf.curve_fit(exponential, x, y, p0=[4, 0.4, 1.5])
    times.append(time.time() - start)

print(f"Average fit time: {np.mean(times[1:]):.4f}s")
print("✓ Reusing CurveFit object eliminates compilation overhead")

INFO:nlsq.curve_fit:Starting curve fit n_params=3 | n_data_points=1000 | method=trf | solver=auto | batch_size=None | has_bounds=False | dynamic_sizing=False


INFO:nlsq.least_squares:Starting least squares optimization method=trf | n_params=3 | loss=linear | ftol=1.0000e-08 | xtol=1.0000e-08 | gtol=1.0000e-08


INFO:nlsq.optimizer.trf:Starting TRF optimization (no bounds) n_params=3 | n_residuals=1000 | max_nfev=None


PERFORMANCE:nlsq.optimizer.trf:Iteration iter=0 | cost=163.72333537348317 | grad_norm=1108.3590 | nfev=1


PERFORMANCE:nlsq.optimizer.trf:Iteration iter=1 | cost=5.333929441542062 | grad_norm=23.3586 | step=4.290687590584987 | nfev=2


...

PERFORMANCE:nlsq.least_squares:Timer: optimization elapsed=0.029960s


INFO:nlsq.least_squares:Convergence reason=`ftol` termination condition is satisfied. | iterations=4 | final_cost=4.9910 | elapsed=0.030s | final_gradient_norm=6.4279e-06


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit elapsed=0.103053s


INFO:nlsq.curve_fit:Curve fit completed total_time=0.1031 | final_cost=9.9820 | covariance_warning=False


Average fit time: 0.1131s
✓ Reusing CurveFit object eliminates compilation overhead


## 3. Fixed Array Size**Issue:** Different array sizes trigger recompilation**Solution:** Use `flength` parameter

In [6]:
# Without fixed length (recompiles for each size)
jcf_dynamic = CurveFit()
sizes = [100, 200, 300]

for n in sizes:
    x_test = np.linspace(0, 10, n)
    y_test = 5 * np.exp(-0.5 * x_test) + 2
    start = time.time()
    popt, _ = jcf_dynamic.curve_fit(exponential, x_test, y_test, p0=[4, 0.4, 1.5])
    elapsed = time.time() - start
    print(f"Size {n}: {elapsed:.4f}s")

print("\n⚠️ Each size triggers recompilation")

INFO:nlsq.curve_fit:Starting curve fit n_params=3 | n_data_points=100 | method=trf | solver=auto | batch_size=None | has_bounds=False | dynamic_sizing=False


INFO:nlsq.least_squares:Starting least squares optimization method=trf | n_params=3 | loss=linear | ftol=1.0000e-08 | xtol=1.0000e-08 | gtol=1.0000e-08


INFO:nlsq.optimizer.trf:Starting TRF optimization (no bounds) n_params=3 | n_residuals=100 | max_nfev=None


PERFORMANCE:nlsq.optimizer.trf:Iteration iter=0 | cost=16.313323939646803 | grad_norm=109.9246 | nfev=1


PERFORMANCE:nlsq.optimizer.trf:Iteration iter=1 | cost=0.036080255088984145 | grad_norm=2.0706 | step=4.290687590584987 | nfev=2


...

PERFORMANCE:nlsq.least_squares:Timer: optimization elapsed=0.840932s


INFO:nlsq.least_squares:Convergence reason=`gtol` termination condition is satisfied. | iterations=4 | final_cost=1.4434e-26 | elapsed=0.841s | final_gradient_norm=3.5445e-12


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit elapsed=1.191318s


INFO:nlsq.curve_fit:Curve fit completed total_time=1.1913 | final_cost=2.8868e-26 | covariance_warning=False


Size 300: 1.3317s

⚠️ Each size triggers recompilation


## 4. GPU vs CPU Performance**GPU advantage:** Increases with dataset size

In [7]:
if devices[0].platform == 'gpu':
    print("🚀 GPU Performance Test")
    sizes = [1000, 10000, 100000]

    for n in sizes:
        x_large = np.linspace(0, 10, n)
        y_large = 5 * np.exp(-0.5 * x_large) + 2 + np.random.normal(0, 0.1, n)
        start = time.time()
        popt, _ = curve_fit(exponential, x_large, y_large, p0=[4, 0.4, 1.5])
        elapsed = time.time() - start
        print(f"  {n:>6} points: {elapsed:.4f}s")
else:
    print("💻 CPU mode - GPU would provide 100-300x speedup")

INFO:nlsq.curve_fit:Starting curve fit n_params=3 | n_data_points=1000 | method=trf | solver=auto | batch_size=None | has_bounds=False | dynamic_sizing=False


INFO:nlsq.least_squares:Starting least squares optimization method=trf | n_params=3 | loss=linear | ftol=1.0000e-08 | xtol=1.0000e-08 | gtol=1.0000e-08


🚀 GPU Performance Test


INFO:nlsq.optimizer.trf:Starting TRF optimization (no bounds) n_params=3 | n_residuals=1000 | max_nfev=None


PERFORMANCE:nlsq.optimizer.trf:Iteration iter=0 | cost=164.9711459371021 | grad_norm=1117.3224 | nfev=1


...

PERFORMANCE:nlsq.least_squares:Timer: optimization elapsed=0.297763s


INFO:nlsq.least_squares:Convergence reason=`ftol` termination condition is satisfied. | iterations=4 | final_cost=499.6795 | elapsed=0.298s | final_gradient_norm=2.6924e-05


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit elapsed=0.570361s


INFO:nlsq.curve_fit:Curve fit completed total_time=0.5704 | final_cost=999.3590 | covariance_warning=False


  100000 points: 0.7055s


## 🔑 Key Takeaways1. **Reuse CurveFit objects** to avoid recompilation2. **Use fixed array size** when processing multiple datasets3. **GPU acceleration** scales with dataset size4. **Warmup first fit** to compile functions5. **Profile before optimizing** - measure actual bottlenecks---

## 🔗 Next Steps- [Large Dataset Demo](large_dataset_demo.ipynb) - Memory optimization- [GPU Deep Dive](../03_advanced/gpu_optimization_deep_dive.ipynb) - Advanced GPU techniques- [Advanced Features](advanced_features_demo.ipynb) - Callbacks, robust fitting---